In [3]:
import xml.etree.ElementTree as ET
from datetime import datetime
from googlesearch import search
import time
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import string
import os
from unidecode import unidecode

c:\users\ramon\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
def search_yahoo(Query):

    
    time.sleep(0.33)
    
    url = "http://search.yahoo.com/search?p=%s"
    r = requests.get(url % Query) 
    soup = BeautifulSoup(r.text)
    hrefs = [a['href'] for a in soup.find_all('a', href=True)]
    hrefs = [href for href in hrefs if 'yahoo' not in hrefs]

    new_hrefs = [href for href in hrefs if "yahoo" not in href and '#' not in href and 'bingj' not in href and '.pdf' not in href]
    
    return new_hrefs

def extract_urls(hrefs, site, last_name, functie = '', organisatie = ''):
    
    possible = False
    
    last_names = set()
    last_names.add(last_name)
    
    if '-' in last_name:
        parts = last_name.split('-')
        for i in parts:
            last_names.add(i)
            last_names.add(unidecode(i))
            
    if ' ' in last_name:
        last_names.add(last_name.replace(' ', ''))
        last_names.add(last_name.replace(' ', '-'))
        last_names.add(last_name.replace(' ', '_'))
        last_names.add(last_name.replace(' ', '+'))
        
    if site in ['twitter.com', 'nl.linkedin.com', 'nl.wikipedia.org']:
        for href in hrefs[:5]:
            
            href_copy = href
            href = href.lower()
            if 'bingj' in href:
                continue

            if site == 'nl.linkedin.com':
                if '/in/' not in href_copy:
                    continue

            if site == 'nl.wikipedia.org':
                if '/wiki/' not in href:
                    continue

            if site == 'twitter.com':
                if '/status' in href or '/hashtag' in href:
                    continue
                    
            for ln in last_names:
                if ln.lower() in href:
                    if site in href:
                        
                        if site == 'nl.wikipedia.org':
                            return href_copy
                    
                        return href
    
    if site not in ['twitter.com', 'nl.linkedin.com', 'nl.wikipedia.org']:

        possibilities = [url for url in hrefs[:10] for ln in last_names if ln.lower() in url]
        possibilities2 = [url for url in hrefs[:10] if functie.lower() in url]
        pos = set(possibilities).union(set(possibilities2))

        pos = [p for p in pos if not 'nieuws' in p and not 'news' in p]

        if pos != []:  
            ws = min(pos, key=len)
            if not '.pdf' in ws:
                return ws


        p = [h for h in hrefs if 'dagelijks-bestuur' in h.lower()]

        if len(p) == 1:
            return p[0]

        elif len(p) > 1:
            p2 = [i for i in p if 'leden' in i or 'overzicht' in i or 'samenstelling' in i]
            if len(p2) == 1:
                return p2[0]

            elif len(p2) > 1:
                return ws

    return ''

In [281]:
def find_firstName(hrefs, last_name, initials):
    
    initials = initials.replace('.','').lower()

    last_name = last_name.lower()
    
    last_names = set()
    last_names.add(last_name)
    last_names.add(unidecode(last_name))

    if ' ' in last_name:
        last_names.add(last_name.replace(' ', ''))
        last_names.add(last_name.replace(' ', '-'))
        last_names.add(last_name.replace(' ', '_'))
        last_names.add(last_name.replace(' ', '+'))
    
    possible_firstNames = []
    wrong_names = []
    
    for href in hrefs:

        href = href.lower()
        href = href.split('/')
        for part in href:
            found_last_names = [last_name for last_name in last_names if last_name in part]
            if found_last_names:
                part = part.split(found_last_names[0])[0]
                part = re.split(r"[-_]", part)
                part = [p for p in part if p != '']
                if len(part) > 0:
                    if part[-1][0] == initials[0]:
                        possible_firstNames.append(part[-1])
                    else:
                        wrong_names.append(part[-1])
                        if len(part) > 1:
                            if part[-2][0] == initials[0]:
                                possible_firstNames.append(part[-2] + '-' + part[-1])
                    
                    
    
    
    names = [naam for naam in possible_firstNames if any(letter in 'aeiouy' for letter in naam) and len(naam) > 1]

    names_with_dash = {name.replace('-', ''): name for name in names if '-' in name}
    names = [names_with_dash.get(name, name) for name in names]
    
    names = [name for name in names if re.sub(r'[^\w\s]', '', name) != initials]
    
    if names != []:
        first_option = Counter(names).most_common(1)
        if first_option[0][1] > 1:
            return first_option[0][0].title()
        
    return ''

In [286]:
with open(f"Social_Media_Gemeente.json", "r") as file:
        json_ws = json.load(file)
        
for k,v in json_ws.items():
    if v['dc_publisher_name'] == 'Gemeente Rotterdam':
        if v['foaf_firstName'] == '':
            print(v['foaf_name'])
            query = f'"{v["foaf_name"]}" {v["dc_publisher_name"]} {v["foaf_function_type"]}'
            hrefs = search_yahoo(query)
            find_firstName(hrefs, v['foaf_lastName'], v['foaf_initials'])


A. Aboutaleb
Ahmed
P.J.H.D. Verkoelen

M.W. Vlasblom
Megawati
R.C.G. Segers-Hoogendoorn
Rene
T.J. Vonk
Tjalling
A. Maassen
Agnes
N.D.Z.R. Mohamed-Hoesein

A. Oostdijk

J. Bokhove
Judith
B. van Drunen
Bart
R.T.J. van der Velden
Ruud
G.J. Engberts

T. Coskun
Theo
I. Elfilali
Imane
P.J.P. Lansink-Bastemeijer
Pascal
V.J.M. Roozen
Vincent
R.A.C.J. Simons
Robert
I.C.M. Broeders

P.J.H.D. Verkoelen

M. de Jong
Martin
M.W. Vlasblom

R.C.G. Segers-Hoogendoorn
Rene
T.J. Vonk
Tjalling
F. Elbay

T. de Haan

A. Maassen
Agnes
J. Nunnely
Joan
I. van Wifferen
Ingrid
N. Akhatab
Naoufal
N.D.Z.R. Mohamed-Hoesein

E. Rehimzadeh

S. Soytekin
Serkan
A. Oostdijk

M. Biljoen

J. Bokhove
Judith
A. Kockelkoren
Astrid
M. Morkoc
Mina
L. Vlieger
Larissa
C. van Aalst

S.M. Bonte

V. Bruin
Vanessa
S. Ceulemans

I. Coenradie

B. van Drunen

T. Roskam
Thomas
B. van Schaik

M. Verhoef

J. de Waard

S. van de Peppel
Sabrina
R.T.J. van der Velden
Ruud
G.J. Engberts

R. Moti
Richard
S. Reitema
Sarah
D. Yildirim
Duygu
T. C

In [288]:
def update_social_media(Organisatie):
    
    if (os.path.exists(f"Social_Media_{Organisatie}.json")) == False:
        return None
    
    with open(f"Social_Media_{Organisatie}.json", "r") as file:
        json_data = json.load(file)
        
    for k, v in json_data.items():
        
        query = f'"{v["foaf_name"]}" {v["dc_publisher_name"]} {v["foaf_function_type"]} {Organisatie}'

        
        print(query)
                
        linkedin_hrefs = search_yahoo('site:nl.linkedin.com ' + query)
        if linkedin_hrefs == []:
            linkedin_hrefs = search_yahoo('linkedin ' + query)
        
        foi_linkedin = extract_urls(linkedin_hrefs, 'nl.linkedin.com', v['foaf_lastName'])
        

        twitter_hrefs = search_yahoo('site:twitter.com ' + query)
        
        foi_twitter = extract_urls(twitter_hrefs,'twitter.com', v['foaf_lastName'], '', Organisatie)

        wikipedia_hrefs = search_yahoo('site:nl.wikipedia.org ' + query)

        foi_wikipedia = extract_urls(wikipedia_hrefs, 'nl.wikipedia.org',v['foaf_lastName'])
        
        if v['foaf_firstName'] == '':
            query = f'"{v["foaf_name"]}" {v["dc_publisher_name"]} {v["foaf_function_type"]}'
            hrefs = search_yahoo(query)
            foaf_firstName = find_firstName(hrefs, v['foaf_lastName'], v['foaf_initials'])
            json_data[k]['foaf_firstName'] = foaf_firstName
            print(foaf_firstName)
            
        print(foi_linkedin)
        print(foi_twitter)
        print(foi_wikipedia)
        
        json_data[k]['foi_linkedin'] = foi_linkedin
        json_data[k]['foi_twitter'] = foi_twitter
        json_data[k]['foi_wikipedia'] = foi_wikipedia
        
        # omdat ministeries altijd dezelfde website hebben is dit op een andere manier op te lossen
        if Organisatie != 'Ministerie':
            
            site = v['website']
        
            if site.startswith("http://"):
                # Split the URL asfter "http://"
                site = site.split("://", 1)[1]
            elif site.startswith("https://"):
                # Split the URL after "https://"
                site = site.split("://", 1)[1]

            website_hrefs = search_yahoo(f"site:{site} " + k)
            foi_website = extract_urls(website_hrefs, site, v['foaf_lastName'], k.split(' ')[-1])
            
            json_data[k]['foaf_workplaceHomepage'] = foi_website   
            
            print(foi_website)
        
        json_data[k]['up_to_date'] = True
        
    return json_data

json_data = update_social_media('Waterschap')

"M.J.J. Jacobs" Aa en Maas Dijkgraaf Waterschap
Mario
https://nl.linkedin.com/in/mariojacobs1972
https://twitter.com/jacobsmario
https://nl.wikipedia.org/wiki/Mario_Jacobs
https://www.aaenmaas.nl/bestuur/dagelijks-bestuur/dagelijks-bestuur/mario-jacobs/
"P. Sennema" Aa en Maas Secretaris-directeur Waterschap
Pieter
https://nl.linkedin.com/in/pietersennema


https://www.aaenmaas.nl/bestuur/directieteam/pieter-sennema/
"J.J. Sylvester" Amstel, Gooi en Vecht Dijkgraaf Waterschap
Joyce
https://nl.linkedin.com/in/dr-joyce-sylvester-4b2ab785
https://twitter.com/jjsylvester1
https://nl.wikipedia.org/wiki/Joyce_Sylvester
https://www.agv.nl/over-ons/het-bestuur-in-beeld/joyce-sylvester/
"G. Nottelman" Amstel, Gooi en Vecht Secretaris-directeur Waterschap
Gritta
https://nl.linkedin.com/in/grittanottelman


https://www.agv.nl/over-ons/het-bestuur-in-beeld/gritta-nottelman/
"C.J.G.M. de Vet" Brabantse Delta Dijkgraaf Waterschap



https://nl.wikipedia.org/wiki/Kees_Jan_de_Vet
https://www.brabantse

In [361]:
twitters = [k['foi_twitter'] for i, k in json_data.items() if k['foi_twitter'] != '']
c_twitters = [k for k, v in Counter(twitters).items() if v != 1]
print(c_twitters)
for k, v in json_data.items():
    if v['foi_twitter'] in c_twitters:
        json_data[k]['foi_twitter'] = ''
    

[]


In [359]:
Organisatie = 'Waterschap'

Json = json.dumps(json_data, indent=4)

with open(f"Social_Media_{Organisatie}.json", "w") as file:
    file.write(Json)

In [360]:
with open(f"Social_Media_{Organisatie}.json", "r") as file:
        sm_w = json.load(file)
    
    
count = 0
for k,v in sm_w.items():
    print(k)
    print(v['foi_twitter'])
    
    if v['foi_twitter'] == '':
        count += 1 
        
print(len(sm_w)-count)
print(count)

    

Mark Rutte Algemene Zaken Minister-president, minister van Algemene Zaken
https://twitter.com/minpres/
Wopke Hoekstra Buitenlandse Zaken Minister van Buitenlandse Zaken, viceminister-president
https://twitter.com/wbhoekstra/
Liesje Schreinemacher Buitenlandse Zaken Minister voor Buitenlandse Handel en Ontwikkelingssamenwerking
https://twitter.com/lschreinemacher
Paul Huijts Buitenlandse Zaken Secretaris-generaal

Kajsa Ollongren Defensie Minister
https://twitter.com/defensiemin
Christophe van der Maat Defensie Staatssecretaris
https://twitter.com/cavandermaat
Gea van Craaikamp Defensie Secretaris-generaal

Marc Gazenbeek Defensie Plaatsvervangend secretaris-generaal

Ernst Kuipers Volksgezondheid, Welzijn en Sport Minister van Volksgezondheid, Welzijn en Sport

Conny Helder Volksgezondheid, Welzijn en Sport Minister voor Langdurige Zorg en Sport
https://twitter.com/ministerlzs
Maarten van Ooijen Volksgezondheid, Welzijn en Sport Staatssecretaris van Volksgezondheid, Welzijn en Sport

B

J. Goedhart Hoogheemraadschap De Stichtse Rijnlanden Secretaris-directeur

https://www.linkedin.com/in/simonesteendijk/?originalSubdomain=nl

Net als P.F.C.W. van der Broeck Waterschap Limburg Voorzitter Waterschap
    
Deze zijn inmiddels geen dijkgraaf meer.

Dirt Siert Schoonman en Geert Jan ten Brink hebben wel twitter.